In [ ]:
import os
import math
import torch
import torch.nn as nn
import requests
import numpy as np
from PIL import Image
from tqdm import tqdm  
from abc import abstractmethod
import matplotlib.pyplot as plt
from torchvision import transforms
import torch.nn.functional as F
import torch_optimizer as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader


from DDIM_C import GaussianDiffusion
from ldm.modules.diffusionmodules.openaimodel import UNetModel


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"GPU Name: {device_name}")
else:
    print("No GPU available, using CPU.")

GPU Name: NVIDIA GeForce RTX 4090


In [ ]:
#Step 1 : 

transform2 = transforms.Compose([
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

class PalmDataset(Dataset):
    def __init__(self, palm_dir, label_dir, transform=None):
        self.palm_dir = palm_dir
        self.label_dir = label_dir
        self.transform = transform
        
        self.palm_images = sorted([f for f in os.listdir(palm_dir) 
                                if os.path.isfile(os.path.join(palm_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        self.label_images = sorted([f for f in os.listdir(label_dir) 
                                    if os.path.isfile(os.path.join(label_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))])


    def __len__(self):
        return len(self.palm_images)

    def __getitem__(self, idx):
        palm_path = os.path.join(self.palm_dir, self.palm_images[idx])
        label_path = os.path.join(self.label_dir, self.label_images[idx])
        palm_image = Image.open(palm_path).convert("RGB")
        label_image = Image.open(label_path).convert("RGB")
        
        palm_image = palm_image.resize((256, 256))
        label_image = label_image.resize((64, 64))

        if self.transform:
            palm_image = self.transform(palm_image)
            label_image = self.transform(label_image)

        return palm_image, label_image



real_image_folder = './datasets/palm2bezier/train/trainA'
bezier_image_folder='./datasets/palm2bezier/test/binary_images_fake_bezier'

dataset = PalmDataset(real_image_folder, bezier_image_folder, transform=transform2)

def custom_collate_fn(batch):
    palm_images = torch.stack([item[0] for item in batch])
    label_images = torch.stack([item[1] for item in batch])
    return palm_images, label_images

train_loader = DataLoader(dataset, 
                          batch_size=32, 
                          shuffle=True, 
                          num_workers=16, 
                          pin_memory=True, 
                          collate_fn=custom_collate_fn)

In [ ]:
#Step 2 : 
ckpt_path = 'vqmodel_checkpoint.ckpt'
checkpoint = torch.load(ckpt_path, map_location=device, weights_only=True)


class VQModel(torch.nn.Module):
    def __init__(self, ddconfig, embed_dim=3,n_embed=8192):
        super().__init__()
        from taming.modules.diffusionmodules.model import Encoder, Decoder
        # from taming.modules.vqvae.quantize import VectorQuantizer2 as VectorQuantizer
        self.encoder = Encoder(**ddconfig)
        # self.decoder = Decoder(**ddconfig)
        # self.quantize = VectorQuantizer(n_embed, embed_dim, beta=0.25,
        #                                 remap=None,
        #                                 sane_index_shape=False)
        self.quant_conv = torch.nn.Conv2d(ddconfig["z_channels"], embed_dim, 1)
        # self.post_quant_conv = torch.nn.Conv2d(embed_dim, ddconfig["z_channels"], 1)

    def encode(self, x):  
        h = self.quant_conv(self.encoder(x))
        return h

    # def decode(self, x, force_not_quantize=False):  
    #     if not force_not_quantize:
    #         quant, emb_loss, info = self.quantize(x)
    #     else:
    #         quant = x
    #     dec = self.decoder(self.post_quant_conv(quant))
    #     return dec


vq_model = VQModel(  
    ddconfig={
        'double_z': False,
        'z_channels': 3,
        'resolution': 256,
        'in_channels': 3,
        'out_ch': 3,
        'ch': 128,
        'ch_mult': [1, 2, 4],
        'num_res_blocks': 2,
        'attn_resolutions': [],
        'dropout': 0.0
    },
    embed_dim=3,
    n_embed=8192
)


if 'model_state_dict' in checkpoint:
    state_dict = checkpoint['model_state_dict']
elif 'state_dict' in checkpoint:
    state_dict = checkpoint['state_dict']
else:
    state_dict = checkpoint  

filtered_state_dict = {k: v for k, v in state_dict.items() if k in vq_model.state_dict()}
vq_model.load_state_dict(filtered_state_dict, strict=False)

vq_model = vq_model.to(device)
for param in vq_model.parameters():
    param.requires_grad = False

vq_model.eval()

Working with z of shape (1, 3, 64, 64) = 12288 dimensions.


VQModel(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down): ModuleList(
      (0): Module(
        (block): ModuleList(
          (0-1): 2 x ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
        (attn): ModuleList()
        (downsample): Downsample(
          (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
        )
      )
      (1): Module(
        (block): ModuleList(
          (0): ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): 

In [ ]:
#Step 3 : 
unet_config = {
    "image_size": 64,
    "in_channels": 6,
    "out_channels": 3,
    "model_channels": 224,
    "attention_resolutions": [8, 4, 2],
    "num_res_blocks": 2,
    "channel_mult": [1, 2, 3, 4],
    "num_head_channels": 32,
}



unet_model = UNetModel(**unet_config)


checkpoint_path = 'ddim_c.ckpt'  


pretrained_dict = torch.load(checkpoint_path,weights_only = True)


unet_model.load_state_dict(pretrained_dict)

print("Pretrained weights loaded successfully.")

unet_model.to(device)
unet_model.train()

UNetModel(
  (time_embed): Sequential(
    (0): Linear(in_features=224, out_features=896, bias=True)
    (1): SiLU()
    (2): Linear(in_features=896, out_features=896, bias=True)
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): Conv2d(6, 224, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-2): 2 x TimestepEmbedSequential(
      (0): ResBlock(
        (in_layers): Sequential(
          (0): GroupNorm32(32, 224, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Conv2d(224, 224, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (h_upd): Identity()
        (x_upd): Identity()
        (emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=896, out_features=224, bias=True)
        )
        (out_layers): Sequential(
          (0): GroupNorm32(32, 224, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Dropout(p=0, inplace=False)
          (3): Conv2d(224, 224, kernel_size=(3, 3), 

In [ ]:
#Step 4 : 
timesteps = 1000
gaussian_diffusion = GaussianDiffusion(timesteps=timesteps,
        beta_schedule='linear',
        linear_start = 0.0015,
        linear_end= 0.0155)

In [ ]:
#Step 5 : 
optimizer = torch.optim.AdamW(unet_model.parameters(), lr=1e-06)


cosine_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-7)

global_step = 0
num_epochs = 10

for epoch in tqdm(range(num_epochs), desc='Epochs'):
    for palm_images, label_images in train_loader:
        optimizer.zero_grad()
        
        palm_images = palm_images.to(device)
        label_images = label_images.to(device)
        
        batch_size = palm_images.shape[0]
        
        with torch.no_grad():
            latent = vq_model.encode(palm_images).detach()
            
        t = torch.randint(0, timesteps, (batch_size,), device=device).long()
        
        loss = gaussian_diffusion.train_losses(unet_model, latent, t, label_images)
        
        loss.backward() 
        optimizer.step()
            
        global_step += 1
    
    
    cosine_scheduler.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    

    if (epoch + 1) % 5 == 0:
        torch.save(unet_model.state_dict(), 'ddim_c.ckpt')
        print("Model parameters saved to ddim_c.ckpt")


Epochs:   1%|          | 1/100 [04:06<6:46:29, 246.36s/it]

Epoch [1/100], Loss: 0.6326


Epochs:   2%|▏         | 2/100 [08:11<6:41:33, 245.85s/it]

Epoch [2/100], Loss: 0.3956


Epochs:   3%|▎         | 3/100 [12:17<6:37:16, 245.74s/it]

...

Epoch [54/100], Loss: 0.1636


Epochs:  55%|█████▌    | 55/100 [3:45:15<3:04:12, 245.60s/it]

Epoch [55/100], Loss: 0.1502


Epochs:  56%|█████▌    | 56/100 [3:49:20<3:00:04, 245.55s/it]

Epoch [56/100], Loss: 0.2019
